In [3]:
import os
import heudiconv
from glob import glob
import pydicom as dicom
import nibabel.nicom.csareader as csareader
import os
import pandas as pd

C:\Users\Patricio Viera\anaconda3\envs\Python3-6\lib\site-packages\ipykernel_launcher.py:5: UserWarning: The DICOM readers are highly experimental, unstable, and only work for Siemens time-series at the moment
Please use with caution.  We would be grateful for your help in improving them
  """


In [4]:
base_dir = '/Users/Patricio Viera/Documents/GitHub'
subj_dir = '/Mattfeld_REVL-000-vCAT-021-S1/scans/10-fMRI_REVL_Study_1/resources/DICOM/files'
data_dir = base_dir + subj_dir

#heudiconv -d ../sourcedata/Mattfeld_REVL-000-vCAT-{subject}-S1/*/*/*/*/*/* -s 021 -c none -f convertall -o /Users/amattfel/Documents/GitHub/mattfeld_2020/dset

#heudiconv -d C:/Users/Patricio Viera/Documents/Mattfeld_REVL-000-vCAT-{subject}-S1/*/*/*/*/*/* -s 021 -c none -f convertall -o 'C:\Users\Patricio Viera\Documents\GitHub\mattfeld_2020'

In [5]:
# Step 1 - run heudiconv
#heudiconv -d C:/Users/Patricio Viera/Documents/Mattfeld_REVL-000-vCAT-{subject}-S1/*/*/*/*/*/* -s 021 -c none -f convertall -o 'C:\Users\Patricio Viera\Documents\GitHub\mattfeld_2020/dset'
#/Users/amattfel/Documents/GitHub/mattfeld_2020/dset/.heudiconv/021/info
path2info_file = os.path.join(base_dir, 'mattfeld_2020', 'dset', '.heudiconv/021/info')
dicominfo_df = pd.read_csv(path2info_file + '/dicominfo.tsv', sep="\t")
limited_dicominfo_df = dicominfo_df[['series_id', 'series_files', 'dim1', 'dim2', 'dim3', 'dim4', 'series_description']]

limited_dicominfo_df


,series_id,series_files,dim1,dim2,dim3,dim4,series_description
0,1-localizer_32ch,3,512,512,3,1,localizer_32ch
1,2-T1w_MPR_vNav_setter,32,32,32,32,1,T1w_MPR_vNav_setter
2,3-T1w_MPR_vNav,145,32,32,32,145,T1w_MPR_vNav_setter
3,4-T1w_MPR_vNav,176,256,256,176,1,T1w_MPR_vNav
4,5-T1w_MPR_vNav,176,256,256,176,1,T1w_MPR_vNav
5,6-fMRI_DistortionMap_PA,66,100,100,66,1,fMRI_DistortionMap_PA
6,7-fMRI_DistortionMap_AP,66,100,100,66,1,fMRI_DistortionMap_AP
7,8-fMRI_REVL_ROI_loc_1,304,100,100,66,304,fMRI_REVL_ROI_loc_1
8,9-fMRI_REVL_ROI_loc_2,304,100,100,66,304,fMRI_REVL_ROI_loc_2
9,10-fMRI_REVL_Study_1,355,100,100,66,355,fMRI_REVL_Study_1


In [6]:
def create_key(template, outtype=('nii.gz',), annotation_classes=None):
    if template is None or not template:
        raise ValueError('Template must be a valid format string')
    return template, outtype, annotation_classes

In [11]:
def infotodict(seqinfo):
    """Heuristic evaluator for determining which runs belong where

    allowed template fields - follow python string module:

    item: index within category
    subject: participant id
    seqitem: run number during scanning
    subindex: sub index within group
    """
    # Paths done in BIDS format##
    data = create_key('run{item:03d}')
    info = {data: []}
    last_run = len(seqinfo)
    
    for s in seqinfo:
        """
        The namedtuple `s` contains the following fields:

        * total_files_till_now
        * example_dcm_file
        * series_id
        * dcm_dir_name
        * unspecified2
        * unspecified3
        * dim1
        * dim2
        * dim3
        * dim4
        * TR
        * TE
        * protocol_name
        * is_motion_corrected
        * is_derived
        * patient_id
        * study_description
        * referring_physician_name
        * series_description
        * image_type
        """

        info[data].append(s.series_id)
    return info
